- [GBDT 回顾](#GBDT-回顾)

- [XGBoost](#XGBoost)
  - [系统设计](#XGBoost-系统设计)
- [算法实现](#算法实现)
- [LightGBM](#LightGBM)

# GBDT 回顾

一般树模型具有以下优点：

- 可解释性强
- 可处理混合类型特征
- 具有伸缩不变性（不用归一化特征）
- 有特征组合的作用
- 可自然地处理缺失值
- 对异常点鲁棒
- 有特征选择作用
- 可扩展性强，容易并行

也有一些缺点：

- 缺乏平滑性（回归预测时输出值只能输出有限的若干种数值）
- 不适合处理高维稀疏数据

GBDT 更进一步， 通过放弃在参数空间内寻找最优 $w$，直接在函数空间内寻找最优 $f$ ，拟合残差替换掉了降低误差，再结合提升法站在巨人肩膀上前进的特点 ，GBDT 具有以下突出优点：

- 预测时，不同树间可以并行加速

- 稠密数据集上，泛化能力很强

- 采用决策树作为弱分类器使得GBDT模型具有较好的解释性和鲁棒性，能够自动发现特征间的高阶关系

但是提升法和回归模型的局限 GBDT 也继承了不少：

- 高维稀疏数据集上表现不如 SVM，DNN
- 处理数值特征的效果明显好于文本分类之流的离散特征
- 为了效率，往往需要归一化

- 训练时，树与树之间只能串行训练，很难优化加速

于是自然的想法就是针对算法本身和暴露的缺点，努力进行原理和工程上的优化，于是 XGBoost 出现了。

# XGBoost

XGBoost 算是对 GBDT 算法思想工业化实现的结果，充满了效率的考量。

宏观上 XGBoost 依旧是加法模型，可以和传统 GBDT 实现一样采用 CART 回归树作为基分类器。
$$
\hat{y}_{i}=\phi\left(\mathbf{x}_{i}\right)=\sum_{k=1}^{K} f_{k}\left(\mathbf{x}_{i}\right), \quad f_{k} \in \mathcal{F} \\\mathcal{F}=\left\{f(\mathbf{x})=w_{q(\mathbf{x})}\right\}\left(q: \mathbb{R}^{m} \rightarrow T, w \in \mathbb{R}^{T}\right)
$$
$q(x)$ 表示将 $x$ 分配到叶子上，$w$ 是叶子对应分数，$w_{q(x)}$ 表示模型对样本的预测值。回归树输出的连续值可以用于回归、分类和排序各式任务中，分类时要借助 sigmod 函数转换成概率。

- 增加正则项，防止过拟合

GBDT 的目标函数为 $\mathcal{L}(\Theta) =  L\left(y, f_{m-1}\left(x\right)+ T\left(x ; \Theta_{m}\right)\right)$ ，XGBoost 添加了正则化项，以平衡经验误差和结构误差。正则化相当于添加了模型参数的先验分布，L2 正则是高斯分布，多数绝对值很小，L1正则多数值为 0
$$
\mathcal{L}(\Theta)=L+\Omega(f)= \sum_{i} l\left(\hat{y}_{i}, y_{i}\right)+ \sum_m\Omega\left(f_{m}\right) \\
\Omega(f)=\gamma T+\frac{1}{2} \lambda\|w\|^{2}
$$
XGBoost 选择了叶子数 $T$ 和权重的 L2 正则项表示树的复杂度

- 二阶泰勒展开

GBDT 是单纯的负梯度更新，即只使用一阶导数，典型损失函数为
$$
\mathcal{L}^{(t)}=\sum_{i=1}^{n} l\left(y_{i}, \hat{y}_{i}^{(t-1)}+f_{t}\left(\mathbf{x}_{i}\right)\right)+\Omega\left(f_{t}\right)
$$
XGBoost 选择对损失函数进行二阶泰勒展开进行替换
$$
\mathcal{L}^{(t)} \simeq \sum_{i=1}^{n}\left[l\left(y_{i}, \hat{y}^{(t-1)}\right)+g_{i} f_{t}\left(\mathbf{x}_{i}\right)+\frac{1}{2} h_{i} f_{t}^{2}\left(\mathbf{x}_{i}\right)\right]+\Omega\left(f_{t}\right)
$$
其中 $g_{i}=\partial_{\hat{y}^{(t-1)}} l\left(y_{i}, \hat{y}^{(t-1)}\right)$,$h_{i}=\partial_{\hat{y}^{(t-1)}}^{2} l\left(y_{i}, \hat{y}^{(t-1)}\right)$ ,去掉常数项，带入 $f,\Omega$ 
$$
\begin{aligned} \widetilde{L}^{(t)} &=\sum_{i=1}^{n}\left[g_{i} f_{t}\left(x_{i}\right)+\frac{1}{2} h_{i} f_{t}^{2}\left(x_{i}\right)\right]+\Omega\left(f_{t}\right) \\ &=\sum_{i=1}^{n}\left[g_{i} w_{q\left(x_{i}\right)}+\frac{1}{2} h_{i} w_{q\left(x_{i}\right)}^{2}\right]+\gamma T+\lambda \frac{1}{2} \sum_{j=1}^{T} w_{j}^{2} \end{aligned}
$$
注意到前面是 $\sum_{i=1}^{n}$，是对样本累加，而后面 $ \sum_{j=1}^{T}$ 则是对叶子结点累加。为统一二者，引入叶子结点的样本集合$I_{j}=\left\{i | q\left(x_{i}\right)=j\right\}$
$$
\begin{aligned} \tilde{L}^{(t)} &=\sum_{j=1}^{T}\left[\left(\sum_{i \in I_{j}} g_{i}\right) w_{j}+\frac{1}{2}\left(\sum_{i \in I_{j}} h_{i}+\lambda\right) w_{j}^{2}\right]+\gamma T \\ &=\sum_{j=1}^{T}\left[G_{j} w_{j}+\frac{1}{2}\left(H_{j}+\lambda\right) w_{j}^{2}\right]+\gamma T \end{aligned}
$$
当样本分配打分策略 $w_{q(x)}$ 确定时，上式求导令其为 0，得
$$
w_{j}^{*}=-\frac{G_{j}}{H_{j}+\lambda} \\
\tilde{L}^{*}=-\frac{1}{2} \sum_{j=1}^{T} \frac{G_{j}^{2}}{H_{j}+\lambda}+\gamma T
$$

- 损失衰减幅度作为增益

但打分策略的确定实际就是要确认树结构，即生成一棵树。如果继承决策树思想，选择“增益”大的进行分裂，那么增益是什么呢？

注意到 $\tilde{L}^{*}$ 中后一部分是叶子数，而前一部分可以视作每个叶子结点带来的经验损失，所以可以选择“减小损失幅度”作为增益标准，增益越大，分裂后 $\tilde{L}^{*}$ 越小
$$
G a i n=\frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{\left(G_{L}+G_{R}\right)^{2}}{H_{L}+H_{R}+\lambda}-\gamma
$$
具体执行上，当然可以精确计算每个`<特征取值, 收益>` 对，然后选最大的。但是这样计算效率不高，XGBoost 给出了近似算法，只考察取值分位点的收益，而不用遍历所有值。可以在训练树前就定好分位点，在分裂前重新给定分位点

而 XGBoost 也不是简单按样本数计算分位点，而是以二阶导数值作为权重进行计算的，为什么这么处理呢？可以将 $\mathcal{L}^{(t)}$ 整理成下面的形式
$$
\mathcal{L}^{(t)} = \sum_{i=1}^{n} \frac{1}{2} h_{i}\left(f_{t}\left(\mathbf{x}_{i}\right)-\frac{g_{i}}{h_{i}}\right)^{2}+\Omega\left(f_{t}\right)+\text { constant }
$$
可以看出二阶导有加权的效果

- 缺失值处理

传统的GBDT没有设计对缺失值进行处理，XGBoost 能够自动学习出特征出现缺失值时，默认的结点分裂方向。假设缺失值都分配给了左边，计算一下损失减幅，再假设都分配给了右边，计算相应损失，两相比较

- 其他特点（区别于GBDT）

XGBoost 借鉴了随机森林，支持对数据（行、列）采样。

XGBoost 支持自定义损失函数，但得二阶可导。

Shrinkage，缩减学习速率，迭代次数增多，有正则化作用

## XGBoost 系统设计

- Column Block

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-20_xgboost-column-block.png)

特征预排序，以 column block 结构存于内存；同时存储样本索引；块内数据以 CSC 格式存储

这一结构加速了 split finding 过程，建树前排序一次，后续分裂时直接根据索引获取梯度信息

> 预分类算法：
>
> - 对于每个节点，遍历所有特征
> - 对于每个特征，根据特征值分类样例
> - 进行线性扫描，根据当前特征的基本信息增益，确定最优分割
> - 选取所有特征分割结果中最好的一个

- 缓存感知

column block 按特征值大小顺序存储，势必造成梯度信息是分散的，内存不连续访问，降低缓存命中

对此可以预缓存到 buffer 再排序，或是调整块大小


# 算法实现

In [1]:
import numpy as np

In [2]:
%load_ext watermark
%watermark -v -m -p ipywidgets,numpy,pandas

CPython 3.7.3
IPython 7.6.1

ipywidgets 7.5.0
numpy 1.16.4
pandas 0.24.2

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


**构造基分类器**

In [3]:
class DNode:
    """树结点"""
    
    def __init__(self, feature_i=None, threshold=None, value=None,
                 yes_subtree=None, no_subtree=None):
        self.feature_i = feature_i
        self.threshold = threshold
        self.value = value
        self.yes_subtree = yes_subtree
        self.no_subtree = no_subtree   

In [4]:
def divide_on_feature(X, feature_i, threshold):
    '''给定划分阈值，返回按特征值分类后的数据集'''
    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        split_func = lambda sample:sample[feature_i] >= threshold
    else:
        split_func = lambda sample:sample[feature_i] == threshold
    
    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])
    
    return np.array([X_1, X_2])

In [5]:
class XGBoostRegressionTree(object):
    """XGBoost回归树模型"""
    
    def __init__(self, min_split_num=2, min_impurity=1e-7, loss=None):
        
        self.root = None
        
        # 停止条件，样本数少于该值不进行进一步分割
        self.min_split_num = min_split_num
        
        # 停止条件，当划分带来的增益小于该值时停止生成
        self.min_impurity = min_impurity
        
        # 标签是否经过 one-hot 编码，默认没有(one-dim)
        self.one_dim = False
        
        # 梯度提升
        self.loss = loss
        
    def fit(self, X_train, y_train, loss=None):
        self.one_dim = len(np.shape(y_train)) == 1
        self.root = self.__build_tree(X_train, y_train)
        self.loss = loss
        
    def __split(self, y):
        '''y 内一半是实际值 y_true,另一半是预测值 y_pred'''
        col = int(np.shape(y)[1]/2)
        y, y_pred = y[:, :col],y[:, col:]
        return y, y_pred
    
    def __gain(self, y, y_pred):
        '''单节点带来的损失，gradient^2/Hession'''
        nominator = np.power((y * self.loss.gradient(y, y_pred)).sum(), 2)
        denominator = self.loss.hess(y, y_pred).sum()
        
        return (nominator / denominator) * 0.5
    
    def __gain_by_taylor(self, y, y1, y2):
        '''分裂损失计算'''
        y, y_pred = self.__split(y)
        y1, y1_pred = self.__split(y1)
        y2, y2_pred = self.__split(y2)
        
        true_gain = self.__gain(y1, y1_pred)
        false_gain = self.__gain(y2, y2_pred)
        gain = self.__gain(y, y_pred)
        
        return true_gain + false_gain - gain
    
    def __approximate_update(self, y):
        '''牛顿法拟合残差'''
        y, y_pred = self.split(y)
        
        gradient = np.sum(y * self.loss.gradient(y, y_pred), axis=0)
        hessian = np.sum(self.loss.hess(y, y_pred), axis=0)
        
        return gradient / hessian
    
    def __build_tree(self, X_train, y_train):
        """递归创建基分类器"""
        
        max_impurity = 0
        best_criteria = None  # 最优划分特征
        best_sets = None  # 最优划分形成的子集集合
        
        # 拼接成习惯的训练集形式
        if len(np.shape(y_train)) == 1:
            y_train = np.expand_dims(y_train, axis=1)
        train_set = np.concatenate((X_train, y_train),axis=1)
        
        n_samples, n_features = np.shape(X_train)
        
        if n_samples > self.min_split_num: # 停止条件之一
            
            for feature_i in range(n_features):
                
                # 当前特征可取哪些值
                feature_values = np.expand_dims(X_train[:,feature_i], axis=1)
                unique_values = np.unique(feature_values)
                
                # 计算以当前特征为划分标准时对应的误差
                for threshold in unique_values:
                    
                    X_y1, X_y2 = divide_on_feature(train_set, feature_i, threshold)
                    
                    if len(X_y1) > 0 and len(X_y2) > 0:  # 还有划分的必要
                        y1 = X_y1[:, n_features:]
                        y2 = X_y2[:, n_features:]
                        
                        impurity = self.__gain_by_taylor(y_train, y1, y2)
                        
                        if impurity > max_impurity:
                            max_impurity = impurity
                            best_criteria = {"feature_i":feature_i, "threshold":threshold}
                            best_sets = {
                                "yes_X": X_y1[:, : n_features],
                                "yes_y": X_y1[:, n_features : ],
                                "no_X": X_y2[:, : n_features],
                                "no_y": X_y2[:, n_features : ]
                            }
                            
                # 还值得继续生成树
                if max_impurity > self.min_impurity:
                
                    yes_subtree = self.__build_tree(best_sets["yes_X"],best_sets["yes_y"])
                    no_subtree = self.__build_tree(best_sets["no_X"],best_sets["no_y"])
                    return DNode(feature_i=best_criteria["feature_i"], threshold=best_criteria["threshold"],
                            yes_subtree=yes_subtree, no_subtree=no_subtree)
        
        # 停止划分，已经成为叶子结点了，计算此时的预测输出
        leaf_value = self.__approximate_update(y_train)
            
        return DNode(value=leaf_value)
    
    
    
    def predict_value(self, x, tree=None):
        '''树tree对样本x的预测,递归实现'''
        
        if tree is None:
            tree = self.root
        
        # 抵达叶子结点，给出预测
        if tree.value is not None:
            return tree.value
        
        # 还在内部结点，选择前进方向
        feature_value = x[tree.feature_i]
        goto = tree.yes_subtree
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value < tree.threshold:
                goto = tree.no_subtree
        elif feature_value != tree.threshold:
            goto = tree.no_subtree
        
        return self.predict_value(x, goto)
    
    def predict(self, X_test):
        y_pred = [self.predict_value(sample) for sample in X_test]
        return y_pred
    

**定义损失计算方法**

In [6]:
class Sigmoid(object):
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))
    
    def gradient(self, x):
        return self.__call__(x) * (1 - self.__call_(x))

In [7]:
class LogisticLoss(object):
    '''对数损失'''
    def __init__(self):
        sigmoid = Sigmoid()
        self.log_func = sigmoid
        self.log_grad = sigmoid.gradient
        
    def loss(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
        p = self.log_func(y_pred)
        return y * np.log(p) + (1 - y) * np.log(1-p)
    
    def gradient(self, y, y_pred):
        p = self.log_func(y_pred)
        return -(y-p)
    
    def hess(self, y, y_pred):
        p = self.log_func(y_pred)
        return p * (1-p)

**XGBoost**

In [8]:
def to_category(x, n_col=None):
    '''独热编码'''
    if not n_col:
        n_col = np.amax(x)+1
    one_hot = np.zeros((x.shape[0],n_col))
    one_hot[np.arange(x.shape[0]),x] = 1
    return one_hot

In [9]:
class XGBoost(object):
    '''XGBoost 分类器'''
    
    def __init__(self, n_estimators=200, learning_rate=0.001, min_split_samples=2,
                min_impurity=1e-7):
        self.n_estimator = n_estimators
        self.learning_rate = learning_rate
        self.min_split_samples = min_split_samples
        self.min_impurity = min_impurity
        
        self.loss = LogisticLoss()
        self.trees = []
        for _ in range(n_estimators):
            tree = XGBoostRegressionTree(
                    min_split_num = self.min_split_samples,
                    min_impurity = self.min_inpurity,
                    loss = self.loss)
            self.trees.append(tree)
            
    def fit(self, X_train, y_train):
        y_train = to_category(y_train)
        
        y_pred = np.zeros(np.shape(y_train))
        
        for i in range(self.n_estimators):
            tree = self.trees[i]
            y_and_pred = np.concatenate((y_train, y_pred), axis=1)
            tree.fit(X_train, y_and_pred)
            update_pred = tree.predict(X_train)
            
            y_pred -= np.multiply(self.learning_rate, update_pred)
            
    def predict(self, X_test):
        y_pred = None
        for tree in self.trees:
            update_pred = tree.predict(X_test)
            if y_pred is None:
                y_pred = np.zeros_like(update_pred)
            y_pred -= np.multiply(self.learning_rate, update_pred)
        
        # Softmax
        y_pred = np.exp(y_pred) / np.sum(np.exp(y_pred), axis=1,keepdims=True)
        
        return np.argmax(y_pred,axis=1)

# LightGBM

LightGBM 是 XGBoost 的升级版，同样是基于树的梯度提升算法框架，训练速度更快，内存占用更低，准确率旗鼓相当，可以处理大规模数据。有这么几个改进：

- 直方图算法

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-20_lightGBM-histograms.png)

分桶思想，类似对年龄的处理，把连续的浮点特征值离散化成k个整数，同时构造一个宽度为k的直方图，即分出了 k 个桶。遍历数据，按所处区间放入不同桶内，各离散值分别累计计数。完成后根据离散值遍历寻找最优分割点。

通过分桶降低了内存占用，8 bit就可以离散成 256 个桶，同时减少了生成树时计算增益的计算量，从 O(#data) 降低为 O(#bins) 后者是一个常数

- 直方图作差加速

一个叶子的直方图可以通过其父亲的直方图减去其兄弟结点的直方图得到，加速一倍

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-22_hinstgram_subtraction.png)

仅需选取`#data`较小的叶子建好一个直方图，就能在 O(#bins）的时间内得到兄弟节点的直方图

- 树生成策略

XGBoost 是层次优先生成（level-wise tree growth），每层所有结点都分裂，然后剪枝

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-20_xgboost-level-wise-growth.png)

LightGBM 是深度优先生成（leaf-wise tree growth），每次只生成分裂收益最大的结点，但容易过拟合，通过最大树深来限制

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-22_xgboost-leaf-wise-growth.png)

- 并行优化

》**特征并行**

每个 worker 保留所有数据集，在其拥有的特征子集上寻找最优分裂点，之后彼此通信确认全局最优分割点，每个 worker 根据全局最优分裂点分割数据

好处是免去了广播数据索引，降低了通信量。缺点是这么做本身并不会降低分裂计算的工作量，同时如果数据量较大时，存储所有数据的开销有些高

》**数据并行**

传统做法：

1. 水平切分数据，每个worker只有部分数据
2. 每个worker根据本地数据统计局部直方图
3. 合并所有局部直方图得到全局直方图
4. 根据全局直方图进行节点分裂

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-20_traditional-data-parall.png)

缺点显而易见，网络通信开销太大了。如果是端到端的通信算法，每个 worker 的通信量为 O(#machine * #feature * #bin)，如果是集体通信算法，每个 worker 通信量为 O(2 * #feature * #bin)

LightGBM 做法：

1. 特征集分为互斥的若干部分，不同的 worker 合并不同特征的局部直方图
2. 借助直方图作差，只需要广播一个结点的直方图，其它直方图就能自行调整

通信量只需要 O(0.5 * #feature * #bin)

》**投票并行**

每个worker中选出top k个分裂特征，然后将每个worker选出的k个特征进行汇总，并选出全局分裂特征，进行数据分裂。 

>- 基于梯度的单边采样（GOSS） --降低所需数据量
>
> 在过滤数据样例寻找分割值时，LightGBM 使用的是全新的技术：基于梯度的单边采样（ **G**radient-based **O**ne **S**ide **S**ampling ） 。在每一次迭代前，利用了GBDT中的样本梯度和误差的关系，对训练样本进行采样: 对误差大（梯度绝对值大）的数据保留；对误差小的数据采样一个子集，但给这个子集的数据一个权重，让这个子集可以近似到误差小的数据的全集。这么采样出来的数据，既不损失误差大的样本，又在减少训练数据的同时不改变数据的分布，从而实现了在几乎不影响精度的情况下加速了训练。
>
>- 孤立特征捆绑 (EFB)--降低特征量
>
>在特征维度很大的数据上，特征空间一般是稀疏的。利用这个特征，我们可以无损地降低GBDT算法中需要遍历的特征数量，更确切地说，是降低构造特征直方图（训练GBDT的主要时间消耗）需要遍历的特征数量。在稀疏的特征空间中，很多特征是exclusive的（即在同一个样本里，这一组特征里最多只有一个特征不为0）。每一组exclusive feature都可以无损地合并成一个“大特征”。构造直方图的时候，遍历一个“大特征”可以得到一组exclusive feature的直方图。这样只需要遍历这些“大特征”就可以获取到所有特征的直方图，降低了需要遍历的特征量。


---------------

作者：Daniel Meng

GitHub：[LibertyDream](https://github.com/LibertyDream)

Blog: [明月轩](https://libertydream.github.io/)